In [11]:
from web3 import Web3

# 1. Connect to Ganache
ganache_url = "http://127.0.0.1:7545"  # Change to 8545 if needed
web3 = Web3(Web3.HTTPProvider(ganache_url))

# 2. Check connection
if web3.is_connected():
    print("✅ Connected to Ganache successfully!")
else:
    print("❌ Connection failed. Make sure Ganache is running.")

# 3. Convert deployed contract address to checksum
contract_address = web3.to_checksum_address("0xb03734fcb403dfb1225f5ad728c45a07d5afd88d")

# 4. ABI (from Remix / your PDF)
abi = [
    {
        "inputs": [{"internalType": "address", "name": "device", "type": "address"}],
        "name": "authorizeDevice",
        "outputs": [],
        "stateMutability": "nonpayable",
        "type": "function"
    },
    {
        "inputs": [{"internalType": "address", "name": "device", "type": "address"}],
        "name": "revokeDevice",
        "outputs": [],
        "stateMutability": "nonpayable",
        "type": "function"
    },
    {
        "inputs": [
            {"internalType": "string", "name": "deviceID", "type": "string"},
            {"internalType": "string", "name": "dataType", "type": "string"},
            {"internalType": "string", "name": "value", "type": "string"}
        ],
        "name": "storeData",
        "outputs": [],
        "stateMutability": "nonpayable",
        "type": "function"
    },
    {
        "inputs": [],
        "stateMutability": "nonpayable",
        "type": "constructor"
    },
    {
        "inputs": [{"internalType": "address", "name": "", "type": "address"}],
        "name": "authorizedDevices",
        "outputs": [{"internalType": "bool", "name": "", "type": "bool"}],
        "stateMutability": "view",
        "type": "function"
    },
    {
        "inputs": [
            {"internalType": "string", "name": "deviceID", "type": "string"},
            {"internalType": "uint256", "name": "index", "type": "uint256"}
        ],
        "name": "getDataByIndex",
        "outputs": [
            {"internalType": "string", "name": "", "type": "string"},
            {"internalType": "string", "name": "", "type": "string"},
            {"internalType": "uint256", "name": "", "type": "uint256"}
        ],
        "stateMutability": "view",
        "type": "function"
    },
    {
        "inputs": [{"internalType": "string", "name": "deviceID", "type": "string"}],
        "name": "getDataCount",
        "outputs": [{"internalType": "uint256", "name": "", "type": "uint256"}],
        "stateMutability": "view",
        "type": "function"
    },
    {
        "inputs": [],
        "name": "owner",
        "outputs": [{"internalType": "address", "name": "", "type": "address"}],
        "stateMutability": "view",
        "type": "function"
    }
]

# 5. Load the contract
contract = web3.eth.contract(address=contract_address, abi=abi)

# 6. Set the default sender (1st Ganache account)
web3.eth.default_account = web3.eth.accounts[0]

print(f"✅ Smart contract loaded at: {contract_address}")
print(f"Using sender address: {web3.eth.default_account}")

# 7. Authorize sender address
try:
    tx = contract.functions.authorizeDevice(web3.eth.default_account).transact()
    web3.eth.wait_for_transaction_receipt(tx)
    print("✅ Device authorized successfully!")
except Exception as e:
    print("❌ Authorization failed:", e)

# 8. Store dummy IoT data
try:
    tx_hash = contract.functions.storeData("TEST001", "Temperature", "22.5°C").transact({
        'from': web3.eth.default_account,
        'gas': 1000000
    })
    web3.eth.wait_for_transaction_receipt(tx_hash)
    print("✅ Dummy data stored on blockchain!")
except Exception as e:
    print("❌ Transaction failed:", e)

# 9. Retrieve stored data
try:
    count = contract.functions.getDataCount("TEST001").call()
    print(f"📦 Total Records for TEST001: {count}")

    if count > 0:
        record = contract.functions.getDataByIndex("TEST001", 0).call()
        print("🔎 First Stored Record:", record)
except Exception as e:
    print("❌ Retrieval failed:", e)


✅ Connected to Ganache successfully!
✅ Smart contract loaded at: 0xB03734FCb403DFb1225f5Ad728C45a07D5AFD88d
Using sender address: 0xc0D384c3dD0f715D426f92ffd6049E3A9e72E3f2
✅ Device authorized successfully!
✅ Dummy data stored on blockchain!
📦 Total Records for TEST001: 1
🔎 First Stored Record: ['Temperature', '22.5°C', 1748727028]
